In [1]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117 > /dev/null
!pip install deepfloyd-if==1.0.1 > /dev/null
!pip install xformers==0.0.16 > /dev/null
!pip install git+https://github.com/openai/CLIP.git --no-deps > /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-d83stiz9


In [2]:
!pip list | grep deepfloyd
!pip list | grep xformers
!pip list | grep torch
!nvidia-smi | head -12

deepfloyd-if                  1.0.1
xformers                      0.0.16
torch                         1.13.1+cu117
torchaudio                    0.13.1+cu117
torchdata                     0.6.1
torchsummary                  1.5.1
torchtext                     0.15.2
torchvision                   0.14.1+cu117
Sun Jun  4 22:22:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8

In [3]:
import os
import torch
os.environ['FORCE_MEM_EFFICIENT_ATTN'] = "1"
import sys
from deepfloyd_if.modules import IFStageI, IFStageII, StableStageIII
from deepfloyd_if.modules.t5 import T5Embedder
from deepfloyd_if.pipelines import dream, style_transfer, super_resolution, inpainting
import torch.nn.functional as F
import random
import torchvision.transforms as T
import numpy as np
import requests
from PIL import Image
import torch
import re

FORCE_MEM_EFFICIENT_ATTN= 1 @UNET:QKVATTENTION


In [4]:
from huggingface_hub import login

login()

In [5]:
%%time

device = 'cuda:0'
if_I = IFStageI('IF-I-L-v1.0', device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1104: FutureWarning: The `force_filename` parameter is deprecated as a new caching system, which keeps the filenames as they are on the Hub, is now in place.
  warnings.warn(


CPU times: user 5.28 s, sys: 5.4 s, total: 10.7 s
Wall time: 1min 52s


In [1]:
%%time

if_II = IFStageII('IF-II-L-v1.0', device=device)

NameError: ignored

In [ ]:
%%time

# recommendation: dont use or use carefully if_III_kwargs in any pipes with "StableStageIII", 
# as it is not native IF-III:
if_III = StableStageIII('stable-diffusion-x4-upscaler', device=device)

In [ ]:
%%time

t5 = T5Embedder(device=device)

# I. Dream 

### I-a. Core method. Dream with prompt using all three cascades: stage-I, stage-II, stage-III: 
### 64-->256-->1024px

In [ ]:
prompt = 'ultra close-up color photo portrait of rainbow owl with deer horns in the woods'

count = 4

result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[prompt]*count,
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
    },
)
if_I.show(result['I'], size=3)
if_I.show(result['II'], size=6)
if_I.show(result['III'], size=14)

In [ ]:
prompt = "a photo of a kangaroo wearing an orange hoodie and blue sunglasses standing in front of the eiffel tower holding a sign that says 'very deep learning'"

count = 4

result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[prompt]*count,
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
    },
)
if_I.show(result['I'], size=3)
if_I.show(result['II'], size=6)
if_I.show(result['III'], size=14)

### I-b. Dream with control of style: 

"style_prompt" - text

"positive_mixer" - [0, 1.0], recommended 0.4 or 0.25..0.5

In [ ]:
prompt = 'ultra close-up color photo portrait of rainbow owl with deer horns in the woods'
style_prompt = 'in style lego'
# style_prompt = 'in style cubism'

count = 4

result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[prompt]*count,
    style_prompt=[style_prompt]*count,
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
        "positive_mixer": 0.4,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
        "positive_mixer": 1.0,
    },
)
if_I.show(result['III'])

### I-c. Dream with control style and adding style in main prompt

"style_prompt" - text

"positive_mixer" - [0, 1.0], recommended 0.25 or 0.25..0.5

In [ ]:
prompt = 'ultra close-up color photo portrait of rainbow owl with deer horns in the woods'
style_prompt = 'in style lego'
# style_prompt = 'in style cubism'

count = 4

result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[f'{style_prompt}, {prompt}']*count,
    style_prompt=[style_prompt]*count,
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
        "positive_mixer": 0.4,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
        "positive_mixer": 1.0,
    },
)
if_I.show(result['III'])

### I-d. Dream with different prompts in one batch

In [ ]:
result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[
        "A vibrant professional close-up photo portrait of a brutal dog in a helmet driving the harley davidson motorcycle in the street of the new york city, bokeh, 35mm, golden hour, in style of William Eggleston, fujifilm",
        'colour fashion photo of a Teddy Bear with t-shirt with text "Deep Floyd"',
        "a little green budgie parrot driving small red toy car in new york street, photo",
        "A black background photo of rainbow owl with deer horns, full hd, 4k, detailed picture",
    ],
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
    },
)
if_I.show(result['I'], size=3)
if_I.show(result['II'], size=6)
if_I.show(result['III'], size=9)

In [ ]:
result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[
        "A vibrant professional close-up photo portrait of a brutal dog in a helmet driving the harley davidson motorcycle in the street of the new york city, bokeh, 35mm, golden hour, in style of William Eggleston, fujifilm",
        'colour fashion photo of a Teddy Bear with t-shirt with text "Deep Floyd"',
        "a little green budgie parrot driving small red toy car in new york street, photo",
        "A black background photo of rainbow owl with deer horns, full hd, 4k, detailed picture",
    ],
    style_prompt=[
        "in style cubism",
        "in style lego",
        "in style minecraft",
        "in style origami",
    ],
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
        "positive_mixer": 0.4,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
        "positive_mixer": 1.0,
    },
)
if_I.show(result['III'])

### I-e. Dream with negative prompts - help model, exclude unnecessary concept

In [ ]:
result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[
        "A vibrant professional close-up photo portrait of a brutal dog in a helmet driving the harley davidson motorcycle in the street of the new york city, bokeh, 35mm, golden hour, in style of William Eggleston, fujifilm",
        'colour fashion photo of a Teddy Bear with t-shirt with text "Deep Floyd"',
        "a little green budgie parrot driving small red toy car in new york street, photo",
        "A black background photo of rainbow owl with deer horns, full hd, 4k, detailed picture",
    ],
    style_prompt=[
        "in style cubism",
        "in style lego",
        "in style minecraft",
        "in style origami",
    ],
    negative_prompt=[
        "yellow",
        "yellow",
        "yellow",
        "yellow",
    ],
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
        "positive_mixer": 0.4,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
        "positive_mixer": 1.0,
    },
)
if_I.show(result['III'])

### I-e. Dream with aspect ratio

In [ ]:
prompt = 'ultra close-up color photo portrait of rainbow owl with deer horns in the woods'

count = 2

result = dream(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    prompt=[prompt]*count,
    negative_prompt=['green']*count,
    seed=42,
    aspect_ratio='16:9',
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "smart100",
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": "smart50",
    },
)
if_I.show(result['III'])

# II. Style Transfer

In [ ]:
import requests
from PIL import Image
import torch

img_url = 'https://github.com/CompVis/stable-diffusion/raw/main/assets/stable-samples/img2img/sketch-mountains-input.jpg'
raw_pil_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_pil_image

### II-a. Style Transfer using stage-I cascade with not fully noising

In [ ]:
count = 4

result = style_transfer(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    support_pil_img=raw_pil_image,
    style_prompt=[
        'A fantasy landscape in style lego',
        'A fantasy landscape in style zombie',
        'A fantasy landscape in style origami',
        'A fantasy landscape in style anime',
    ],    
    seed=42,
    if_I_kwargs={
        "guidance_scale": 10.0,
        "sample_timestep_respacing": "10,10,10,10,10,10,10,10,0,0",
        'support_noise_less_qsample_steps': 5,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": 'smart50',
        "support_noise_less_qsample_steps": 5,
    },
)
if_I.show(result['III'], 1, 20)

### II-b. Style Transfer using stage-II cascade (skip stage-I) with high guidance-scale and aug-level

In [ ]:
count = 4

result = style_transfer(
    t5=t5, if_I=None, if_II=if_II, if_III=if_III,
    support_pil_img=raw_pil_image,
    style_prompt=[
        'A fantasy landscape in style lego',
        'A fantasy landscape in style zombie',
        'A fantasy landscape in style origami',
        'A fantasy landscape in style anime',
    ],    
    seed=42,
    if_II_kwargs={
        "guidance_scale": 10.0,
        "sample_timestep_respacing": '100',
        "aug_level": 0.85,

    },
)
if_I.show(result['III'], 1, 20)

### II-c. Style Transfer using double prompting for more control style

In [ ]:
import requests
from PIL import Image
import torch

img_url = 'https://i.kym-cdn.com/entries/icons/facebook/000/000/472/seriouscatcover.jpg'
raw_pil_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_pil_image

In [ ]:
count = 4
prompt = 'white cat'

result = style_transfer(
    t5=t5, if_I=if_I, if_II=if_II, if_III=if_III,
    support_pil_img=raw_pil_image,
    prompt=[prompt]*count,
    style_prompt=[
        f'in style lego',
        f'in style zombie',
        f'in style origami',
        f'in style anime',
    ],
    seed=42,
    if_I_kwargs={
        "guidance_scale": 10.0,
        "sample_timestep_respacing": "10,10,10,10,10,0,0,0,0,0",
        'support_noise_less_qsample_steps': 5,
        'positive_mixer': 0.8,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        "sample_timestep_respacing": 'smart50',
        "support_noise_less_qsample_steps": 5,
        'positive_mixer': 1.0,
    },
)
if_I.show(result['III'], 2, 14)

### III. Inpainting

In [ ]:
raw_pil_image = Image.open('./me/me.jpg').convert('RGB').resize((1024, 1024))
raw_pil_image

In [ ]:
pil_image = raw_pil_image.resize(
    (64, 64), resample=Image.Resampling.BICUBIC, reducing_gap=None
)
img = np.array(pil_image)
img = img.astype(np.float32) / 127.5 - 1
img = np.transpose(img, [2, 0, 1])
img = torch.from_numpy(img).unsqueeze(0)

if_I.to_images(img)[0]

In [ ]:
inpainting_mask = torch.zeros_like(img[0], device='cpu')
inpainting_mask[:, 26:36, 24:34] = 1
inpainting_mask[:, 29:33, 34:36] = 1
inpainting_mask[:, 26:36, 36:44] = 1
inpainting_mask = inpainting_mask.unsqueeze(0)
if_I.to_images((1-inpainting_mask)*img)[0]

In [ ]:
inpainting_mask.shape

In [ ]:
result = inpainting(
    t5=t5, if_I=if_I, 
    if_II=if_II,
    if_III=if_III,
    support_pil_img=raw_pil_image,
    inpainting_mask=inpainting_mask,
    prompt=[
        'blue sunglasses',
        'yellow sunglasses',
        'red sunglasses',
        'green sunglasses',
    ],
    seed=42,
    if_I_kwargs={
        "guidance_scale": 7.0,
        "sample_timestep_respacing": "10,10,10,10,10,0,0,0,0,0",
        'support_noise_less_qsample_steps': 0,
    },
    if_II_kwargs={
        "guidance_scale": 4.0,
        'aug_level': 0.0,
        "sample_timestep_respacing": '100',
    },
)
if_I.show(result['I'], 2, 3)
if_I.show(result['II'], 2, 6)
if_I.show(result['III'], 2, 14)

# IV. Super Resolution [ S O O N]

In [ ]:
def show_superres(raw_pil_img, res_pil_img):
    w, h = res_pil_img.size
    raw_pil_img = raw_pil_img.resize(
        (w, h), resample=getattr(Image, 'Resampling', Image).BICUBIC, reducing_gap=None
    )

    img0 = np.array(raw_pil_img)
    img1 = np.array(res_pil_img)

    combined = np.ones_like(img1)*255
    
    tang = -h/w
    lower_intersection = 0.0
    line_width = 5
    y, x, _ = img1.shape

    yy, xx = np.mgrid[:y, :x]
    img0_positions = xx*tang-line_width//2>(yy-y)
    img1_positions = xx*tang+line_width//2<(yy-y)

    combined[img0_positions] = img0[img0_positions]
    combined[img1_positions] = img1[img1_positions]

    show_img = Image.fromarray(combined)
    return show_img

### III-a. Super Resolution using cascade stage-II and stage-III from low res: 
### 96 (64) --> 384 (256) --> 1024

In [ ]:
img_url = 'https://images.unsplash.com/photo-1597223557154-721c1cecc4b0'
raw_pil_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_pil_image = T.CenterCrop(min(raw_pil_image.size))(raw_pil_image)
raw_pil_image = raw_pil_image.resize((96,96))
raw_pil_image.resize((512,512))

stage-II: 96 --> 384:

In [ ]:
middle_res = super_resolution(
    t5,
    if_III=if_II,
    prompt=['face of beautiful woman, makeup, detailed picture, 4k dslr, best quality'],
    support_pil_img=raw_pil_image,
    img_scale=4.0,
    img_size=96,
    if_III_kwargs={
        'sample_timestep_respacing': 'smart100',
        'aug_level': 0.25,
        'guidance_scale': 4.0,
    },
)

show_superres(raw_pil_image, middle_res['III'][0])

stage-III: 384 --> 1024, no text prompting

In [ ]:
high_res = super_resolution(
    t5,
    if_III=if_III,
    prompt=[''],
    support_pil_img=middle_res['III'][0],
    img_scale=1024/384,
    img_size=384,
    if_III_kwargs={
        'sample_timestep_respacing': 'super100',
        'aug_level': 0.0,
        'guidance_scale': 7.0,
    },
)

show_superres(raw_pil_image, high_res['III'][0])

### better quality:

stage-III: 384 --> 1024, with text prompting


In [ ]:
high_res = super_resolution(
    t5,
    if_III=if_III,
    prompt=['face of beautiful woman, makeup, detailed picture, 4k dslr, best quality'],
    support_pil_img=middle_res['III'][0],
    img_scale=1024/384,
    img_size=384,
    if_III_kwargs={
        'sample_timestep_respacing': 'super100',
        'aug_level': 0.0,
        'guidance_scale': 7.0,
    },
)

show_superres(raw_pil_image, high_res['III'][0])

### III-a. Super Resolution using stage-III from middle res with aspect-ratio, use aug_level to match the distribution of external images
### 384 --> 1024

In [ ]:
img_url = 'https://www.medivet.co.uk/globalassets/assets/shutterstock-and-istock/istock/yawning-cat-good-teeth.jpg'
raw_pil_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_pil_image

In [ ]:
raw_pil_image.size

In [ ]:
_res = super_resolution(
    t5,
    if_III=if_III,
    prompt=['cat, detailed picture, 4k dslr'],
    support_pil_img=raw_pil_image,
    img_scale=1024/384,
    img_size=384,
    if_III_kwargs={
        'sample_timestep_respacing': 'super100',
        'aug_level': 0.2,
        'guidance_scale': 4.0,
    },
)

show_superres(raw_pil_image, _res['III'][0])